## Utils

Conté el codi per descarregar i carregar el model GPT-2 dels repositoris oficials d'OpenAI. A posteriori carregarem els seus pesos, tokenitzador i hiperparàmetre.

In [1]:
import json
import os
import re

import numpy as np
import requests
import tensorflow as tf
from tqdm import tqdm

#Importem el encoder que tenim dins de la carpeta de Modules. (Encoder BPE d'OpenAI GPT-2.)
from modules.encoder import get_encoder

### download_gpt2_files

Funció per descarregar els fitxers de GPT2.

In [2]:
def download_gpt2_files(model_size, model_dir):
    model_full_path = os.path.join(model_dir, model_size)
    if os.path.exists(model_full_path):
        return
    os.makedirs(model_full_path, exist_ok=True)
    #Depenent de quina mida de model volguem fer servir tenim per escollir entre ["124M", "355M", "774M", "1558M"]
    assert model_size in ["124M", "355M", "774M", "1558M"]
    url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    for filename in [
        "checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe",
    ]:
        r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
        r.raise_for_status()

        with open(os.path.join(model_full_path, filename), "wb") as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(
                ncols=100,
                desc="Fetching " + filename,
                total=file_size,
                unit_scale=True,
                unit="b",
            ) as pbar:
                # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)

In [ ]:
#Ens baixem tots els models en les respectives carpetes
model_sizes = ["124M", "355M", "774M", "1558M"]
for model_size in model_sizes:
    download_gpt2_files(model_size=model_size, model_dir="models")

In [3]:
#Amb l'assert ens assegurem que la mida que li passem de model està dins de la llista que definim. 
#Aquesta mida de models és la que ens proporciona OpenAI si no controlem això no ens podrem baixar els fitxers ja que la URL serà incorrecte.
#Si posem un model size que no està dins de la llista que definim obtindrem "AssertionError"
#Definim la variable del directori del model.
model_dir= "models"
model_size="124M"
model_full_path = os.path.join(model_dir, model_size)

In [13]:
assert model_size in model_sizes

In [4]:
#Tenim els noms de fitxers del Model per baixar-nos emmagatzemats en una variable. Són els que anirem a atacar en la URL.
filenames = ["checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe"]

In [5]:
if os.path.exists(model_full_path):
        pass
os.makedirs(model_full_path, exist_ok=True)

In [ ]:
#Creo la carpeta del Directori aqui manualment perquè lo següent funcioni
#Ens definim la URL del Azure Blob Storage Públic d'OpenAI.   
url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
 
#Per cada fitxer del model a descarregar 
for filename in filenames:
    #Atacarem la URL del Blob Públic amb el recurs en qüestió gràcies a la llibreria requests fent una petició HTTP.
    #Stream=True --> Si no es fa servir aquest paràmetre al fer la petició del request tot el contingut de la resposta es descarregarà inmediatament a la memòria abans de processar-lo. Això pot provocar un s elevat de memòria i causar problemes de rendiment si no es disposa de suficient memòria per manejar la càrrega completa del fitxer.
    #Al fer servir el Stream permet processar el contingut per parts (chunks), reduïnt l'ús de memòria millorant el maneig de grans volums de dades
    r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
    #Comprovem que la petició s'ha fet correctament. Si no fa res la resposta és exitosa (codi 200) de lo contrari llença una excepcióo (per ex 400 o 500)
    r.raise_for_status() # --> No retorna res si està OK però és un objecte del tipus: <bound method Response.raise_for_status of <Response [200]>>
    # Creem carpeta, serà "models", i l'arxiu que ens descarreguem (estem creant la ruta completa). Obrirem en mode escritura binaria ja que els arxius que ens baixarem son binaris (com els pesos dels models)
    with open(os.path.join(model_full_path, filename), "wb") as f:
            #Obtindrem la mida total de l'arxiu a descarregar gràcies als encapçalats de la resposta http. Ens servirà per mostrar amb el TQDM el progrés de la descàrrega.
            file_size = int(r.headers["content-length"])
            #Definim la mida de cada chunk o bloc que llegirem i ens baixarem de l'arxiu en cada iteració, s'estableix a 1000 bytes.
            #Si la mida del fitxer és de 10GB i definim el chunksize en 1000, dividim la resposta en 10 parts.
            chunk_size = 1000
            #Amb el TQDM ens permet mostrar el progrés de la descàrrega .
            with tqdm(
                ncols=100, #La amplada de la barra de progrés que es mostrarà en el Output.
                desc="Fetching " + filename, #Descripció en la barra de progrés.
                total=file_size, #El número total d'iteracions esperades, será el mateix que la mida del fitxer.
                unit_scale=True, #Més format de la barra progressiva.
                unit="b", #La mida de la barra de progrés es escala amb la mida de l'arxiu que volem baixar.
            ) as pbar:
                #Gràcies al strem=true del requests permetem que el contingut es descarregui en parts en lloc de carregar tot el contingut de cop en la memòria.
                #Llavors el r.iter_content genera parts de contingut descarregat depenent de la mida especificada pel chunk_size
                for chunk in r.iter_content(chunk_size=chunk_size):
                    #Escribim cada part de contingut del chunk al fitxer.
                    f.write(chunk)
                    #Actualitzem la barra de progrés amb la mida del chunk que hem descarregat.
                    pbar.update(chunk_size)
                    
